In [277]:
import pandas as pd
import random
from time import sleep
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
import numpy as np
import nltk
import os
from sklearn import datasets
import pickle
from nltk.tokenize import word_tokenize
from sklearn.decomposition import NMF, TruncatedSVD
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.neighbors import NearestNeighbors
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.linear_model import RidgeCV
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, roc_curve, auc, fbeta_score
from sklearn.model_selection import learning_curve, KFold, GridSearchCV, cross_val_score
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.dummy import DummyClassifier
from collections import Counter
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction import text
from sklearn.preprocessing import normalize
import re
from unidecode import unidecode

import matplotlib.style as style
%matplotlib inline
plt.style.use('ggplot')

In [278]:
with open('data/cleaned_df.pkl', 'rb') as picklefile:
    df = pickle.load(picklefile)

In [279]:
df.head()

,song_owner,song_title,song_element,verse,artist
0,Childish Gambino,This Is America,[Bridge: Childish Gambino & Young Thug],"We just wanna party, Party just for you, We ju...",Childish Gambino
1,Childish Gambino,This Is America,[Chorus: Childish Gambino],"This is America, Don't catch you slippin' now,...",Childish Gambino
2,Childish Gambino,This Is America,"[Verse 1: Childish Gambino, Blocboy JB, Slim J...","This is America (skrrt, skrrt, woo), Don't cat...",Childish Gambino
3,Childish Gambino,This Is America,[Refrain: Choir & Childish Gambino],"Ooh-ooh-ooh-ooh-ooh, tell somebody, You go tel...",Childish Gambino
4,Childish Gambino,This Is America,"[Chorus: Childish Gambino, Slim Jxmmi & Young ...","This is America (woo, ayy), Don't catch you sl...",Childish Gambino


In [280]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6304 entries, 0 to 6303
Data columns (total 5 columns):
song_owner      6304 non-null object
song_title      6304 non-null object
song_element    6304 non-null object
verse           6304 non-null object
artist          6304 non-null object
dtypes: object(5)
memory usage: 246.3+ KB


In [281]:
model_df = df.copy()

In [282]:
model_df['verse_words'] = model_df['verse'].copy()
model_df['verse_words'] = model_df['verse_words'].str.lower()
model_df['verse_words'] = model_df['verse_words'].apply(lambda x: x.replace(',',''))
model_df['verse_words'] = model_df['verse_words'].apply(lambda x: x.split(' '))

In [283]:
model_df.head()

,song_owner,song_title,song_element,verse,artist,verse_words
0,Childish Gambino,This Is America,[Bridge: Childish Gambino & Young Thug],"We just wanna party, Party just for you, We ju...",Childish Gambino,"[we, just, wanna, party, party, just, for, you..."
1,Childish Gambino,This Is America,[Chorus: Childish Gambino],"This is America, Don't catch you slippin' now,...",Childish Gambino,"[this, is, america, don't, catch, you, slippin..."
2,Childish Gambino,This Is America,"[Verse 1: Childish Gambino, Blocboy JB, Slim J...","This is America (skrrt, skrrt, woo), Don't cat...",Childish Gambino,"[this, is, america, (skrrt, skrrt, woo), don't..."
3,Childish Gambino,This Is America,[Refrain: Choir & Childish Gambino],"Ooh-ooh-ooh-ooh-ooh, tell somebody, You go tel...",Childish Gambino,"[ooh-ooh-ooh-ooh-ooh, tell, somebody, you, go,..."
4,Childish Gambino,This Is America,"[Chorus: Childish Gambino, Slim Jxmmi & Young ...","This is America (woo, ayy), Don't catch you sl...",Childish Gambino,"[this, is, america, (woo, ayy), don't, catch, ..."


In [284]:
from nltk.tokenize import word_tokenize
model_df['tokenize_verse'] = model_df['verse'].copy()
model_df['tokenize_verse'] = model_df['tokenize_verse'].str.lower()
model_df['tokenize_verse'] = model_df['tokenize_verse'].apply(lambda x: word_tokenize(x))

In [285]:
model_df.head()

,song_owner,song_title,song_element,verse,artist,verse_words,tokenize_verse
0,Childish Gambino,This Is America,[Bridge: Childish Gambino & Young Thug],"We just wanna party, Party just for you, We ju...",Childish Gambino,"[we, just, wanna, party, party, just, for, you...","[we, just, wan, na, party, ,, party, just, for..."
1,Childish Gambino,This Is America,[Chorus: Childish Gambino],"This is America, Don't catch you slippin' now,...",Childish Gambino,"[this, is, america, don't, catch, you, slippin...","[this, is, america, ,, do, n't, catch, you, sl..."
2,Childish Gambino,This Is America,"[Verse 1: Childish Gambino, Blocboy JB, Slim J...","This is America (skrrt, skrrt, woo), Don't cat...",Childish Gambino,"[this, is, america, (skrrt, skrrt, woo), don't...","[this, is, america, (, skrrt, ,, skrrt, ,, woo..."
3,Childish Gambino,This Is America,[Refrain: Choir & Childish Gambino],"Ooh-ooh-ooh-ooh-ooh, tell somebody, You go tel...",Childish Gambino,"[ooh-ooh-ooh-ooh-ooh, tell, somebody, you, go,...","[ooh-ooh-ooh-ooh-ooh, ,, tell, somebody, ,, yo..."
4,Childish Gambino,This Is America,"[Chorus: Childish Gambino, Slim Jxmmi & Young ...","This is America (woo, ayy), Don't catch you sl...",Childish Gambino,"[this, is, america, (woo, ayy), don't, catch, ...","[this, is, america, (, woo, ,, ayy, ), ,, do, ..."


In [286]:
model_df['verse'] = model_df['verse'].str.lower()
model_df['verse'] = model_df['verse'].apply(lambda x: x.replace("'",''))
model_df['verse'] = model_df['verse'].apply(lambda x: "".join([c if c.isalnum() else " " for c in x ]))
model_df['verse'] = model_df['verse'].apply(lambda x: x.replace('  ',' '))

In [287]:
stopwords = ["yeah", "uh", "huh", "aw", "oh", "la", "ayy",
            "na", "ha", "ay", "skrrt",
            "em", "bla", "woo", "wu", "ya", "wala", "woop", "wrarrr", "ooh",
            "ba", "duh", "du", "1", "2", "3", "4" ,"5" ,"6", "7", "8", "9", "0"]

for index, row in model_df.iterrows():
    querywords = row['verse'].split()

    resultwords  = [word for word in querywords if word.lower() not in stopwords]
    row['verse'] = ' '.join(resultwords)

model_df.verse[:5]

0    we just wanna party party just for you we just...
1    this is america dont catch you slippin now don...
2    this is america dont catch you slippin now loo...
3    tell somebody you go tell somebody grandma tol...
4    this is america dont catch you slippin now don...
Name: verse, dtype: object

In [288]:
model_df.head()

,song_owner,song_title,song_element,verse,artist,verse_words,tokenize_verse
0,Childish Gambino,This Is America,[Bridge: Childish Gambino & Young Thug],we just wanna party party just for you we just...,Childish Gambino,"[we, just, wanna, party, party, just, for, you...","[we, just, wan, na, party, ,, party, just, for..."
1,Childish Gambino,This Is America,[Chorus: Childish Gambino],this is america dont catch you slippin now don...,Childish Gambino,"[this, is, america, don't, catch, you, slippin...","[this, is, america, ,, do, n't, catch, you, sl..."
2,Childish Gambino,This Is America,"[Verse 1: Childish Gambino, Blocboy JB, Slim J...",this is america dont catch you slippin now loo...,Childish Gambino,"[this, is, america, (skrrt, skrrt, woo), don't...","[this, is, america, (, skrrt, ,, skrrt, ,, woo..."
3,Childish Gambino,This Is America,[Refrain: Choir & Childish Gambino],tell somebody you go tell somebody grandma tol...,Childish Gambino,"[ooh-ooh-ooh-ooh-ooh, tell, somebody, you, go,...","[ooh-ooh-ooh-ooh-ooh, ,, tell, somebody, ,, yo..."
4,Childish Gambino,This Is America,"[Chorus: Childish Gambino, Slim Jxmmi & Young ...",this is america dont catch you slippin now don...,Childish Gambino,"[this, is, america, (woo, ayy), don't, catch, ...","[this, is, america, (, woo, ,, ayy, ), ,, do, ..."


In [289]:
def get_words(df):
    all_words = []
    
    for index, row in df['verse'].iteritems():
        new_words = re.findall(r"\b[a-z']+\b", unidecode(row))
        all_words += new_words
        
    return all_words

In [290]:
model_df.head()

,song_owner,song_title,song_element,verse,artist,verse_words,tokenize_verse
0,Childish Gambino,This Is America,[Bridge: Childish Gambino & Young Thug],we just wanna party party just for you we just...,Childish Gambino,"[we, just, wanna, party, party, just, for, you...","[we, just, wan, na, party, ,, party, just, for..."
1,Childish Gambino,This Is America,[Chorus: Childish Gambino],this is america dont catch you slippin now don...,Childish Gambino,"[this, is, america, don't, catch, you, slippin...","[this, is, america, ,, do, n't, catch, you, sl..."
2,Childish Gambino,This Is America,"[Verse 1: Childish Gambino, Blocboy JB, Slim J...",this is america dont catch you slippin now loo...,Childish Gambino,"[this, is, america, (skrrt, skrrt, woo), don't...","[this, is, america, (, skrrt, ,, skrrt, ,, woo..."
3,Childish Gambino,This Is America,[Refrain: Choir & Childish Gambino],tell somebody you go tell somebody grandma tol...,Childish Gambino,"[ooh-ooh-ooh-ooh-ooh, tell, somebody, you, go,...","[ooh-ooh-ooh-ooh-ooh, ,, tell, somebody, ,, yo..."
4,Childish Gambino,This Is America,"[Chorus: Childish Gambino, Slim Jxmmi & Young ...",this is america dont catch you slippin now don...,Childish Gambino,"[this, is, america, (woo, ayy), don't, catch, ...","[this, is, america, (, woo, ,, ayy, ), ,, do, ..."


In [291]:
get_words(model_df)

['we',
 'just',
 'wanna',
 'party',
 'party',
 'just',
 'for',
 'you',
 'we',
 'just',
 'want',
 'the',
 'money',
 'money',
 'just',
 'for',
 'you',
 'i',
 'know',
 'you',
 'wanna',
 'party',
 'party',
 'just',
 'for',
 'free',
 'girl',
 'you',
 'got',
 'me',
 'dancin',
 'girl',
 'you',
 'got',
 'me',
 'dancin',
 'dance',
 'and',
 'shake',
 'the',
 'frame',
 'we',
 'just',
 'wanna',
 'party',
 'party',
 'just',
 'for',
 'you',
 'we',
 'just',
 'want',
 'the',
 'money',
 'money',
 'just',
 'for',
 'you',
 'you',
 'i',
 'know',
 'you',
 'wanna',
 'party',
 'party',
 'just',
 'for',
 'free',
 'girl',
 'you',
 'got',
 'me',
 'dancin',
 'girl',
 'you',
 'got',
 'me',
 'dancin',
 'dance',
 'and',
 'shake',
 'the',
 'frame',
 'this',
 'is',
 'america',
 'dont',
 'catch',
 'you',
 'slippin',
 'now',
 'dont',
 'catch',
 'you',
 'slippin',
 'now',
 'look',
 'what',
 'im',
 'whippin',
 'now',
 'this',
 'is',
 'america',
 'dont',
 'catch',
 'you',
 'slippin',
 'now',
 'dont',
 'catch',
 'you',
 's

In [292]:
lyrics[:5]

['we', 'just', 'wanna', 'party', 'party']

In [293]:
verses = list(model_df.verse.values)

In [294]:
verses[:10]

['we just wanna party party just for you we just want the money money just for you i know you wanna party party just for free girl you got me dancin girl you got me dancin dance and shake the frame we just wanna party party just for you we just want the money money just for you you i know you wanna party party just for free girl you got me dancin girl you got me dancin dance and shake the frame',
 'this is america dont catch you slippin now dont catch you slippin now look what im whippin now this is america dont catch you slippin now dont catch you slippin now look what im whippin now',
 'this is america dont catch you slippin now look how im livin now police be trippin now this is america guns in my area word my area i got the strap i gotta carry ima go into this ugh this is guerilla ima go get the bag or ima get the pad im so cold like im so dope like we gon blow like straight up',
 'tell somebody you go tell somebody grandma told me get your money black man get your black man get yo

In [295]:
def make_word_dictionary(verses):
    following = {}
    count = {}
    for verse in verses:
        words = verse.split()
        for index, word in enumerate(words[:-1]):
            # ensure the word is in the dictionary
            if word not in following:
                following[word] = []
                count[word] = 0

            # append the next word
            following[word].append(words[index + 1])
            count[word] += 1

        if words:
            last_word = words[-1]
            if last_word not in following:
                following[last_word] = []
                count[last_word] = 0
                
    return count, following

In [296]:
count, following = make_word_dictionary(verses)

In [297]:
def create_bigrams(verses):
    bigrams = {}
    for words in verses:
        words = words.split()
        words = [x for x in words if x]
        for w1, w2, w3 in zip(words[:-2], words[1:-1], words[2:]):
            bigram = (w1,w2)
            if bigram not in bigrams:
                bigrams[bigram] = [w3]
            else:
                bigrams[bigram].append(w3)   
    return bigrams

In [298]:
verses_bigrams = create_bigrams(verses)

In [309]:
def generate_text_from_bigram(following, bigram, num_words, seeds):    
    all_bigrams = list(bigram.keys())
    current_bigram = random.choice(seeds)
    text = [current_bigram]
    for _ in range(num_words - 1):
        if current_bigram in all_bigrams:
            
            next_word = random.choice(bigram[current_bigram])
        else:
            if following[text[-1]]:
                next_word = random.choice(following[text[-1]])
            else:
                continue
        text.append(next_word)
        current_bigram = (current_bigram[1], next_word)
    return ' '.join(text)

In [315]:
seeds = ["bitch", "love", "drugs", "cash", "guns", "club", "friends", "paper", "liquor", "apple",
         "blue", "red", "streets", "people", "brother", "cap"]

seed = ["pizza"]

generated_raps = []
for _ in range(0,100):
    wordlist = generate_text_from_bigram(following, verses_bigrams, 33, seed)
    generated_raps.append(wordlist)
    print(wordlist)
    print('\n')

pizza man up and we back and forth just askin me for my underdog and i dont let me run this house party trippin off my skills mike and himey and some kenzo


pizza with my forefathers even record with gaga too lyricist of the game go hood on and on and haters gonna hate hate theyre always gonna play play but anyway i can tell


pizza man up and i dare you to the tip of your home and she even iffy about you from you cause kissin people ass is grown i put them jeans and nobodys


pizza better find your loving dont stop it now make sure that you dont know me how that shit play superman when them charges brought up tryin to get i walk too much


pizza hut im living in california everybody wanna know how it might look like i practice cause when im dead im on one but they dont know me you and there is nothing


pizza and by default make money from the slums like shhh she got her waitin at my imperfections in awe im free give my last one didnt end like this in im gone


pizza cause i got to g

pizza man up october baby for ironys sake of course gassed off journalistic come at me you aint finna argue bout where im just more ready than petty nowadays just know that place


pizza box all in the game ot no end in sight i could give you my god there must be a better way this time open up your son in it i promise


pizza overdosed on bad bitches crackheads street niggas hood rats ladies and gentlemen childish gambino show a nigga quick that shit was wild goddamnit i think you play with me they got attention


pizza box all in they new v the good pussy that pop preferably the kind you cant believe i will smile and its going long i can visit if we would make my


pizza man up and listen to the next couple bars i want it all it all go play beeotch i look in the hype me im already down breakin my heart even though


pizza hut my dad used to front on me and just to tell me who is that a lesson some people like t j yall aint relatin well let me beat it off


pizza with my ego overly foc